## Heat Transfer from Knee to Ice Water

### Assumptions and declarations

* $t$ is the time variable.
* $T_{w0}$ is the initial temperature of the ice water.
* $T_{sleeve}$ is the temperature of the plastic sleeve and starts at $25 C$
* $T_w$ as the temperature of water flowing throughout the device.
* The device sleeve is a low density polyethylene with thermal coefficient of $0.33 j/(m * K * s)$ at $25 C$
* The skin has a thermal conductivity of $0.31 j/(m * K *s)$
* The water has a thermal conductivity of $0.608 j / (m * k * s)$
* We assume the water has initial temperature of $0 C$
* We assume that the fresh blood perfusing the tissue underneath the skin is constant at temperature $37 C$ and the skin starts at same temperature.
* We assume that the surface area of the capillary bed is the same as the surface area of the knee as a whole (i.e. that fresh blood is constantly being in contact with the tissue.
* We assume that heat transfer from the knee skin to the environment is negligible.
* The sleeve of the device is of size $0.079 m^2$ ($10.85" x 11.34"$)

## Key values to figure out

* We don't need to find out how much water is in the sleeve because we assume it is constant temp throughout.
* We need to figure out how thick the sleeve plastic is between the skin and the sleeve. I'm using the filler variable of .5 mm.
* I am going to use a generic figure for skin depth (2 mm), but I'm not sure that applies to knee skin (https://www.cyh.com/HealthTopics/library/skin.pdf)

## Derivation of the Heat Transfer Equation from Water Through Plastic Sleeve

### Rate of thermal heat transfer:

$\\\dot Q = k * A * \Delta T / d$

$\\\dot Q = k * 0.079 m^2 * (T_w - T_{sleeve})$

### Heat Capacity of Plastic Sleeve:
$\\ Q = m c \Delta T$


In [4]:
# set up dependencies
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
K_ldpe = 0.33
A_sleeve = 0.079
T_wi = 273 #Kelvin
T_sleevei = 298 #Kelvin
c_ldpe = 2100 #http://www.goodfellow.com/E/Polyethylene-Low-Density.html

In [ ]:
def Q()